In [2]:
import unicodecsv as csv
import pandas as pd
import numpy as np
from datetime import datetime as dt

# Data Analysis Process

### Step 1: Get data

In [3]:
# 1a: Find Data

Categorical Data - Any value that if you ran mathematical operation on it, you'd get nonsensical data

Ordinal Data - Any value that if you ran mathematical operation on it, you'd get sensical data

In [4]:
# 1b: Pull in the data

In [5]:
submission_data = pd.read_csv('submissions.csv')
enrollment_data = pd.read_csv('enrollments.csv')
engagement_data = pd.read_csv('engagements.csv')
engagement_data[0:1]

,acct,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed
0,0,2018-01-09,1,11.679374,0,0


### Step 2: Clean Data

In [6]:
# Colum name in engagement_data for 'acct' is not consistent with 'account_key' in other datasets. Change it
# RUN ONLY ONCE!!!!!!!!!!!!!!!
engagement_data.rename(columns={ 'acct': 'account_key' }, inplace=True) 
enrollment_data

,account_key,status,join_date,cancel_date,days_to_cancel,is_enrolled,is_canceled
0,700,canceled,2017-11-10,2017-11-16,6.0,False,True
1,429,canceled,2017-11-10,2018-03-10,120.0,False,True
2,429,canceled,2018-03-10,2018-06-17,99.0,False,True
3,60,canceled,2017-11-10,2018-01-14,65.0,False,True
4,60,canceled,2018-01-14,2018-04-01,77.0,False,True
...,...,...,...,...,...,...,...
1635,312,canceled,2018-04-01,2018-07-02,92.0,True,True
1636,312,current,2018-07-08,NaN,NaN,True,False
1637,818,canceled,2018-01-07,2018-01-12,5.0,True,True
1638,1069,canceled,2018-06-01,2018-06-01,0.0,True,True


In [7]:
np.isnan(enrollment_data['days_to_cancel'])

0       False
1       False
2       False
3       False
4       False
        ...  
1635    False
1636     True
1637    False
1638    False
1639     True
Name: days_to_cancel, Length: 1640, dtype: bool

### Step 3: Answer Questions with Data

In [8]:
print(f'Enrollment Records: {enrollment_data.describe().max().astype("int")[0]}')
print(f'Enrollment Records: {engagement_data.describe().max().astype("int")[0]}')
print(f'Enrollment Records: {submission_data.describe().max().astype("int")[0]}')

Enrollment Records: 1640
Enrollment Records: 136240
Enrollment Records: 3642


In [122]:
unique_enrollments = enrollment_data['account_key'].unique()
unique_engagements = engagement_data['account_key'].unique()
unique_submissions = submission_data['account_key'].unique()
print(f"Unique enrollment records: {len(unique_enrollments)}")
print(f"Unique engagement records: {len(unique_engagements)}")
print(f"Unique submission records: {len(unique_submissions)}")

Unique enrollment records: 1302
Unique engagement records: 1237
Unique submission records: 743


54

In [124]:
outlier_list = []
for idx, row in enrollment_data.iterrows():
    if row['account_key'] not in unique_engagements and row['join_date'] != row['cancel_date']:
        outlier_list.append(row)
        print(f'Outlier #{len(outlier_list)}:',f'\n{outlier_list}', end='\n\n')
print(f'There are {outliers} outlier accounts.')
outlier_list_df = pd.DataFrame(outlier_list)
outlier_list_df

Outlier #1: 
[account_key             1304
status              canceled
join_date         2018-01-10
cancel_date       2018-03-10
days_to_cancel          59.0
is_enrolled             True
is_canceled             True
Name: 1631, dtype: object]

Outlier #2: 
[account_key             1304
status              canceled
join_date         2018-01-10
cancel_date       2018-03-10
days_to_cancel          59.0
is_enrolled             True
is_canceled             True
Name: 1631, dtype: object, account_key             1304
status              canceled
join_date         2018-03-10
cancel_date       2018-06-17
days_to_cancel          99.0
is_enrolled             True
is_canceled             True
Name: 1632, dtype: object]

Outlier #3: 
[account_key             1304
status              canceled
join_date         2018-01-10
cancel_date       2018-03-10
days_to_cancel          59.0
is_enrolled             True
is_canceled             True
Name: 1631, dtype: object, account_key             1304
status 

,account_key,status,join_date,cancel_date,days_to_cancel,is_enrolled,is_canceled
1631,1304,canceled,2018-01-10,2018-03-10,59.0,True,True
1632,1304,canceled,2018-03-10,2018-06-17,99.0,True,True
1639,1101,current,2018-02-25,NaN,NaN,True,False


In [138]:
# Discrepancy in enrollments: 
# only a handfull of the 'is_enrolled' values are true which also align with our outlier accounts 
# these entries with true 'is_enrolled' values are likely test accounts that we can get rid of.

test_accounts = set()
for idx, row in enrollment_data.iterrows():
    if row['is_enrolled']:
        test_accounts.add(row['account_key'])
print(f'{len(test_accounts)} possible test accounts found:')
for acct in test_accounts:
    print(f' Account Key {acct}')
test_accounts_df = pd.DataFrame(test_accounts)
test_accounts_df.rename(columns = { 0:"account_key"}, inplace = True)
test_accounts_df

6 possible test accounts found:
 Account Key 448
 Account Key 312
 Account Key 1069
 Account Key 1101
 Account Key 818
 Account Key 1304


,account_key
0,448
1,312
2,1069
3,1101
4,818
5,1304


In [20]:
def remove_test_accounts(dataset):
    true_records = []
    for idx, row in dataset.iterrows():
        if row['account_key'] not in test_accounts:
            true_records.append(row.to_dict())
    return true_records

true_enrollments = remove_test_accounts(enrollment_data)
true_engagements = remove_test_accounts(engagement_data)
true_submissions = remove_test_accounts(submission_data)

print(f"Enrollment records: {len(enrollment_data)}")
print(f"Engagement records: {len(engagement_data)}")
print(f"Submission records: {len(submission_data)}")
print('\n')
print(f"True enrollment records: {len(true_enrollments)}")
print(f"True engagement records: {len(true_engagements)}")
print(f"True submission records: {len(true_submissions)}")

Enrollment records: 1640
Engagement records: 136240
Submission records: 3642


True enrollment records: 1622
True engagement records: 135656
True submission records: 3634


In [26]:
true_enrollments_df = pd.DataFrame(true_enrollments)
true_engagements_df = pd.DataFrame(true_engagements)
true_submissions_df = pd.DataFrame(true_submissions)
true_enrollment_df

,account_key,status,join_date,cancel_date,days_to_cancel,is_enrolled,is_canceled
0,700,canceled,2017-11-10,2017-11-16,6.0,False,True
1,429,canceled,2017-11-10,2018-03-10,120.0,False,True
2,429,canceled,2018-03-10,2018-06-17,99.0,False,True
3,60,canceled,2017-11-10,2018-01-14,65.0,False,True
4,60,canceled,2018-01-14,2018-04-01,77.0,False,True
...,...,...,...,...,...,...,...
1617,1176,current,2018-08-12,NaN,NaN,False,False
1618,1110,current,2018-08-13,NaN,NaN,False,False
1619,1116,canceled,2018-08-15,2018-08-18,3.0,False,True
1620,874,current,2018-08-22,NaN,NaN,False,False


In [38]:
true_engagements_df

,account_key,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed
0,0,2018-01-09,1,11.679374,0,0
1,0,2018-01-10,2,37.284887,0,0
2,0,2018-01-11,2,53.633746,0,0
3,0,2018-01-12,1,33.489270,0,0
4,0,2018-01-13,1,64.779678,0,0
...,...,...,...,...,...,...
135651,1305,2018-06-14,0,0.000000,0,0
135652,1305,2018-06-15,0,0.000000,0,0
135653,1305,2018-06-16,1,2.720136,0,0
135654,1305,2018-06-17,0,0.000000,0,0


<h4>Find quality accounts in which:</h4>
<ol>
    <li>The account is currently active i.e. is_canceled is False</li>
    <li>days_to_cancel is greater than 7 i.e. the trial period</li>
    <li>The latest vald record for each particular VALID user</li>
</ol>

In [102]:
accounts_in_good_standing = dict()
for row in true_enrollments:
    if row['days_to_cancel'] is not None and not np.isnan(row['days_to_cancel']) and row['cancel_date'] is not None:
        if not row['is_canceled'] or row['days_to_cancel'] > 7:
            if row['account_key'] not in accounts_in_good_standing or row['join_date'] > accounts_in_good_standing.get(row['account_key']):
                accounts_in_good_standing[row['account_key']] = row['join_date']
# print(accounts_in_good_standing)
accounts_in_good_standing_df = pd.DataFrame(accounts_in_good_standing, index=['join_date']).T
print(f'There are {len(accounts_in_good_standing_df)} accounts in good standing.')
accounts_in_good_standing_df



There are 445 accounts in good standing.


,join_date
429,2018-03-10
60,2018-01-14
322,2018-02-12
584,2018-01-14
458,2017-11-10
...,...
1099,2018-05-11
846,2018-05-12
1052,2018-05-12
661,2018-05-12


In [115]:
# How many students were active on the website within the first week?
def remove_free_trials(dataset):
    new_list = []
    for row in dataset:
        if row['account_key'] in accounts_in_good_standing:
            new_list.append(row)
    return new_list

quality_enrollments = remove_free_trials(true_enrollments)
quality_engagements = remove_free_trials(true_engagements)
quality_submissions = remove_free_trials(true_submissions)

quality_enrollments_df = pd.DataFrame(quality_enrollments)
quality_engagements_df = pd.DataFrame(quality_engagements)
quality_submissions_df = pd.DataFrame(quality_submissions)

print(f"Quality enrollments records: {len(quality_enrollments)}")
print(f"Quality engagements records: {len(quality_engagements)}")
print(f"Quality submissions records: {len(quality_submissions)}")

Quality enrollments records: 719
Quality engagements records: 55675
Quality submissions records: 1110


,account_key,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed,has_visited
0,3,2017-11-10,2,136.183600,0,0,1
1,3,2017-11-11,1,11.055044,0,0,1
2,3,2017-11-12,1,34.615231,1,0,1
3,3,2017-11-13,1,406.333269,3,0,1
4,3,2017-11-14,2,91.475528,0,0,1
...,...,...,...,...,...,...,...
55670,1305,2018-06-14,0,0.000000,0,0,0
55671,1305,2018-06-15,0,0.000000,0,0,0
55672,1305,2018-06-16,1,2.720136,0,0,1
55673,1305,2018-06-17,0,0.000000,0,0,0


In [107]:
def engagement_within_first_week(date_joined, date_of_first_engagement):
    time_diff = date_of_first_engagement - pd.to_datetime(date_joined)
    if time_diff.days >= 0 and time_diff.days < 8:
        return True
    
for i in quality_engagements:
    if i['num_courses_visited'] > 0:
        i['has_visited'] = 1
    else:
        i['has_visited'] = 0
    
first_week_engagements = list()
    
for x in quality_engagements:
    if engagement_within_first_week(accounts_in_good_standing[x['account_key']], pd.to_datetime(x['utc_date'])):
        first_week_engagements.append(x)
        
print(f"{len(first_week_engagements)} engagements within the first week.")

3559 engagements within the first week.


In [108]:
first_week_engagements[0]

{'account_key': 3,
 'utc_date': '2017-11-10',
 'num_courses_visited': 2,
 'total_minutes_visited': 136.1835995,
 'lessons_completed': 0,
 'projects_completed': 0,
 'has_visited': 1}

In [120]:
# How many minutes did each students spend online in the first week?
from collections import defaultdict

def compare(dataset, a_key):
    new_data = defaultdict(list)
    for i in dataset:
        account_key = i[a_key]
        new_data[account_key].append(i)
    return new_data

engagements_grouped_by_account = compare(first_week_engagements, 'account_key')

engagements_grouped_by_account[5]

[{'account_key': 5,
  'utc_date': '2018-01-07',
  'num_courses_visited': 1,
  'total_minutes_visited': 59.72671883,
  'lessons_completed': 0,
  'projects_completed': 0,
  'has_visited': 1},
 {'account_key': 5,
  'utc_date': '2018-01-08',
  'num_courses_visited': 0,
  'total_minutes_visited': 0.0,
  'lessons_completed': 0,
  'projects_completed': 0,
  'has_visited': 0},
 {'account_key': 5,
  'utc_date': '2018-01-09',
  'num_courses_visited': 2,
  'total_minutes_visited': 103.5686138,
  'lessons_completed': 0,
  'projects_completed': 0,
  'has_visited': 1},
 {'account_key': 5,
  'utc_date': '2018-01-10',
  'num_courses_visited': 1,
  'total_minutes_visited': 80.18189033,
  'lessons_completed': 0,
  'projects_completed': 0,
  'has_visited': 1},
 {'account_key': 5,
  'utc_date': '2018-01-11',
  'num_courses_visited': 1,
  'total_minutes_visited': 36.24653467,
  'lessons_completed': 0,
  'projects_completed': 0,
  'has_visited': 1},
 {'account_key': 5,
  'utc_date': '2018-01-12',
  'num_cou

In [110]:
def sum_records(dataset, column):
    # dataset = engagements_grouped_by_account
    new_data = dict()
    for k, v in dataset.items():
        sum_ = 0
        for record in v:
            sum_ += record[column]
            new_data[k] = sum_
    return new_data

minutes_per_accounts = sum_records(engagements_grouped_by_account, 'total_minutes_visited')
minutes_per_accounts

{3: 1023.5579746999999,
 5: 370.82494396,
 15: 1071.12533757,
 18: 155.1361575,
 19: 0.0,
 24: 943.1881172000001,
 25: 0.0,
 31: 247.27557963999996,
 34: 138.1789523,
 42: 174.30704300000002,
 43: 88.822038,
 44: 731.72366783,
 45: 979.7454166629999,
 51: 0.0,
 52: 3.227523833,
 53: 43.53851617,
 57: 750.894147533,
 60: 0.0,
 62: 293.3831222,
 64: 589.58194586,
 65: 422.27366,
 67: 1123.02533437,
 69: 0.0,
 72: 883.9906950330001,
 74: 90.48065633,
 76: 17.658779,
 78: 6.609209333,
 79: 416.495983833,
 85: 500.49217067,
 88: 106.16380783,
 90: 4.842514,
 91: 70.37317134,
 92: 715.4277596699999,
 93: 932.7172561699999,
 97: 494.84799353,
 98: 531.4097201300001,
 99: 425.60350947,
 101: 188.92038351,
 105: 573.58079867,
 106: 535.168407333,
 108: 164.270890837,
 110: 1403.143966737,
 111: 169.1595922,
 113: 139.57379017,
 119: 103.90436599,
 123: 0.0,
 130: 0.0,
 132: 1428.3456270000001,
 134: 1098.6832576400002,
 136: 66.30446384,
 138: 46.42543033,
 140: 202.6228267,
 141: 544.9926586,


In [111]:
most_minutes_spent = sorted(minutes_per_accounts.values(), reverse=True)[0]

In [112]:
most_minutes_spent

2183.9339814

In [346]:
for k, v in minutes_per_accounts.items():
    if v == most_minutes_spent:
        print(engagements_grouped_by_account[k])
        break

[{'account_key': 171, 'utc_date': '2018-03-08', 'num_courses_visited': 2, 'total_minutes_visited': 155.9647843, 'lessons_completed': 1, 'projects_completed': 0, 'has_visited': 1}, {'account_key': 171, 'utc_date': '2018-03-09', 'num_courses_visited': 2, 'total_minutes_visited': 188.4147333, 'lessons_completed': 0, 'projects_completed': 0, 'has_visited': 1}, {'account_key': 171, 'utc_date': '2018-03-10', 'num_courses_visited': 1, 'total_minutes_visited': 554.3022598, 'lessons_completed': 0, 'projects_completed': 0, 'has_visited': 1}, {'account_key': 171, 'utc_date': '2018-03-11', 'num_courses_visited': 1, 'total_minutes_visited': 365.7518803, 'lessons_completed': 0, 'projects_completed': 0, 'has_visited': 1}, {'account_key': 171, 'utc_date': '2018-03-12', 'num_courses_visited': 1, 'total_minutes_visited': 307.2154792, 'lessons_completed': 1, 'projects_completed': 0, 'has_visited': 1}, {'account_key': 171, 'utc_date': '2018-03-13', 'num_courses_visited': 1, 'total_minutes_visited': 85.530

In [347]:
# HOMEWORK:

# Take a look at data and come up with 3 good questions and use some of the methods done in this 
# Jupyter Notebook to successfully answer the questions.

In [116]:
quality_engagements_df

,account_key,utc_date,num_courses_visited,total_minutes_visited,lessons_completed,projects_completed,has_visited
0,3,2017-11-10,2,136.183600,0,0,1
1,3,2017-11-11,1,11.055044,0,0,1
2,3,2017-11-12,1,34.615231,1,0,1
3,3,2017-11-13,1,406.333269,3,0,1
4,3,2017-11-14,2,91.475528,0,0,1
...,...,...,...,...,...,...,...
55670,1305,2018-06-14,0,0.000000,0,0,0
55671,1305,2018-06-15,0,0.000000,0,0,0
55672,1305,2018-06-16,1,2.720136,0,0,1
55673,1305,2018-06-17,0,0.000000,0,0,0


### Question 1: Which currently enrolled user has visited the most courses?

In [146]:
def times_visited(dataset, a_key):
    new_data = {}
    for i in dataset:
        account_key = i[a_key]
        if account_key in new_data.keys() and i['has_visited']:
            new_data[account_key] += i['num_courses_visited']
        elif i['has_visited']:
            new_data[account_key] = i['num_courses_visited']
    return new_data

visit_amount_dict = times_visited(quality_engagements, "account_key")
visit_amount_df = pd.DataFrame(visit_amount_dict, index = ["times_visited"]).T
visits = [v for v in visit_amount_dict.values()]

max_visits = max(visits)

for k, v in visit_amount_dict.items():
    if v == max_visits:
        user_with_most_visits = k

print(f'The user with the most visits at {max_visits} visits is account {user_with_most_visits}')
visit_amount_df

The user with the most visits at 203 visits is account 119


,times_visited
3,59
5,72
15,88
18,31
19,36
...,...
1289,6
1292,62
1299,15
1303,30


### Question 2: Which currently enrolled user has completed the most lessons?

In [148]:
def lessons_completed(dataset, a_key):
    new_data = {}
    for i in dataset:
        account_key = i[a_key]
        if account_key in new_data.keys() and i['lessons_completed']:
            new_data[account_key] += i['lessons_completed']
        elif i['lessons_completed']:
            new_data[account_key] = i['lessons_completed']
    return new_data

lessons_completed_dict = lessons_completed(quality_engagements, "account_key")
lessons_completed_df = pd.DataFrame(lessons_completed_dict, index=[lessons_completed]).T
lessons_completed = [v for v in lessons_completed_dict.values()]

max_lessons_completed = max(lessons_completed)

for k, v in lessons_completed_dict.items():
    if v == max_lessons_completed:
        user_with_most_lesson_completions = k

print(f'The user with the most lessons completed at {max_lessons_completed} lessons complete is account {user_with_most_lesson_completions}')
lessons_completed_df

The user with the most lessons completed at 65 lessons complete is account 597


,<function lessons_completed at 0x7fb505bbfd30>
3,42
5,13
15,56
18,6
19,16
...,...
1260,1
1285,1
1292,8
1299,3


### Question 3: What is the average amount of time users spend on each course?

In [157]:

def avg_minutes_per_course(dataset):
    minutes = 0
    courses = 0
    for row, col in dataset.iterrows():
        minutes += col['total_minutes_visited']
        courses += col['num_courses_visited']
    avg = minutes/courses
    return avg

current_users_avg_minutes_per_course = avg_minutes_per_course(quality_engagements_df)
all_users_avg_minutes_per_course = avg_minutes_per_course(engagement_data)

print(f'Current users spend an average of {current_users_avg_minutes_per_course:.2f} minutes on each course.')
print(f'All users have spent an average of {all_users_avg_minutes_per_course:.2f} minutes on each course.')

Current users spend an average of 71.75 minutes on each course.
All users have spent an average of 71.99 minutes on each course.


In [161]:
quality_enrollments

[{'account_key': 429,
  'status': 'canceled',
  'join_date': '2017-11-10',
  'cancel_date': '2018-03-10',
  'days_to_cancel': 120.0,
  'is_enrolled': False,
  'is_canceled': True},
 {'account_key': 429,
  'status': 'canceled',
  'join_date': '2018-03-10',
  'cancel_date': '2018-06-17',
  'days_to_cancel': 99.0,
  'is_enrolled': False,
  'is_canceled': True},
 {'account_key': 60,
  'status': 'canceled',
  'join_date': '2017-11-10',
  'cancel_date': '2018-01-14',
  'days_to_cancel': 65.0,
  'is_enrolled': False,
  'is_canceled': True},
 {'account_key': 60,
  'status': 'canceled',
  'join_date': '2018-01-14',
  'cancel_date': '2018-04-01',
  'days_to_cancel': 77.0,
  'is_enrolled': False,
  'is_canceled': True},
 {'account_key': 60,
  'status': 'current',
  'join_date': '2018-04-01',
  'cancel_date': nan,
  'days_to_cancel': nan,
  'is_enrolled': False,
  'is_canceled': False},
 {'account_key': 322,
  'status': 'canceled',
  'join_date': '2017-11-10',
  'cancel_date': '2018-02-02',
  'day

### Question 4: Which current user has the longest current quality enrollment? 

In [164]:
def longest_current_enrollment(dataset, a_key):
    enrollment_dates = {}
    for i in dataset:
        account_key = i[a_key]
        if not i['is_canceled']:
            enrollment_dates[account_key] = i['join_date']       
    return enrollment_dates
    
current_enrollment_dict = longest_current_enrollment(quality_enrollments, 'account_key')

date_enrolled = [v for v in current_enrollment_dict.values()]  

earliest_enrollment = min(date_enrolled)

for k, v in current_enrollment_dict.items():
    if v == earliest_enrollment:
        user_with_longest_enrollment = k
        
print(f'The user who has been enrolled the longest since {earliest_enrollment} is account {k}')

The user who has been enrolled the longest since 2018-01-14 is account 1217
